In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

In [3]:
# Hyper Parameters
EPOCH = 1
BATCH_SIZE = 64
TIME_STEP = 28 # the height of each picure
INPUT_SIZE = 28 # the number of pixel in each row
LR = 0.01 # learning rate
DOWNLOAD_MNIST = True

In [4]:
train_set = torchvision.datasets.MNIST(
    root='./data/MNIST',
    train=True, # this is training set
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor() # transform to torch.FloatTensor(C x H x W), and normalize to[0.0, 1.0] while training
    ])
)

In [5]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_set, 
    batch_size=BATCH_SIZE,
    shuffle=True
)
train_loader

In [6]:
test_data = torchvision.datasets.MNIST(
    root='./data/MNIST',
    train=False, 
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor() # transform to torch.FloatTensor(C x H x W), and normalize to[0.0, 1.0] while training
    ])
)

test_x = test_data.test_data.type(torch.FloatTensor)[:2000]/255.   # shape (2000, 28, 28) value in range(0,1)
test_y = test_data.test_labels.numpy()[:2000]    # covert to numpy array

C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [7]:
test_x.shape, test_y.shape

(torch.Size([2000, 28, 28]), (2000,))

In [8]:
class RNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.rnn = nn.LSTM(
        input_size=INPUT_SIZE,
        hidden_size=64, # rnn hidden unit
        num_layers=1, # the number of RNN layers
        batch_first=True # input & output shape are default: shape(batch, time_step, input_size)
    )
    
        self.out = nn.Linear(64, 10)
    
    def forward(self, x):
        # x, r_out .shape(batch, time_step, input_step)
        # h_n, h_c .shape(n_layers, batch, hidden_size)
        r_out,(h_n, h_c) = self.rnn(x, None) #LSTM has two hidden states, h_n is bratch line, h_c is main line
        
        #choosing the lastest time point as output (the same value as h_n)
        out = self.out(r_out[:, -1, :])
        return out

In [9]:
rnn = RNN()

In [10]:
rnn

RNN(
  (rnn): LSTM(28, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=10, bias=True)
)

In [11]:
optimizer = optim.Adam(rnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()

In [12]:
# training
for epoch in range(EPOCH):
    for step, (x, y) in enumerate(train_loader):
        b_x = x.view(-1, 28, 28)
        b_y = y
        
        output = rnn(b_x)
        loss = loss_func(output, b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if step % 50 == 0:
            test_output = rnn(test_x)                   # (samples, time_step, input_size)
            pred_y = torch.max(test_output, 1)[1].data.numpy()
            accuracy = float((pred_y == test_y).astype(int).sum()) / float(test_y.size)
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)

Epoch:  0 | train loss: 2.2883 | test accuracy: 0.10
Epoch:  0 | train loss: 0.8821 | test accuracy: 0.59
Epoch:  0 | train loss: 0.8761 | test accuracy: 0.76
Epoch:  0 | train loss: 0.7183 | test accuracy: 0.83
Epoch:  0 | train loss: 0.3006 | test accuracy: 0.87
Epoch:  0 | train loss: 0.2222 | test accuracy: 0.90
Epoch:  0 | train loss: 0.2493 | test accuracy: 0.90
Epoch:  0 | train loss: 0.5102 | test accuracy: 0.90
Epoch:  0 | train loss: 0.1940 | test accuracy: 0.93
Epoch:  0 | train loss: 0.3494 | test accuracy: 0.91
Epoch:  0 | train loss: 0.0745 | test accuracy: 0.93
Epoch:  0 | train loss: 0.1430 | test accuracy: 0.94
Epoch:  0 | train loss: 0.0632 | test accuracy: 0.95
Epoch:  0 | train loss: 0.1455 | test accuracy: 0.94
Epoch:  0 | train loss: 0.1893 | test accuracy: 0.95
Epoch:  0 | train loss: 0.1034 | test accuracy: 0.95
Epoch:  0 | train loss: 0.1091 | test accuracy: 0.95
Epoch:  0 | train loss: 0.1881 | test accuracy: 0.95
Epoch:  0 | train loss: 0.2590 | test accuracy

In [13]:
for step, (x, y) in enumerate(train_loader):
    pass

In [15]:
x.shape

torch.Size([32, 1, 28, 28])

In [16]:
b_x = x.view(-1, 28, 28)

In [17]:
b_x.shape

torch.Size([32, 28, 28])

In [18]:
y.shape

torch.Size([32])

In [19]:
y

tensor([3, 8, 1, 0, 2, 0, 1, 0, 5, 6, 9, 1, 0, 5, 4, 6, 5, 6, 5, 8, 5, 6, 1, 5,
        8, 8, 0, 1, 2, 5, 3, 0])